In [20]:
%pylab inline
from Corrfunc.mocks.DDsmu_mocks import DDsmu_mocks
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM

Populating the interactive namespace from numpy and matplotlib


In [21]:
cosmo = FlatLambdaCDM(Om0=0.3089,H0=100)

In [312]:
pathELG="/Users/rpaviot/eBOSSxDES/faizan/elg/eBOSS_ELG_SGC_pip_v7.dat.fits"
T = Table.read(pathELG)


"""Parent Catalog"""
RAp = T['RA']
DECp = T['DEC']
Zp = T['Z']
Wp = T['WEIGHT_SYSTOT']*T['WEIGHT_FKP']*T['WEIGHT_NOZ']
Dcp = cosmo.comoving_distance(Zp).value




In [314]:
cond = np.where(T['CLUSTERING']==1)
Tc = T[cond]
Tc = Tc[0:1000]

ra = Tc['RA']
dec = Tc['DEC']
z = Tc['Z']
w = Tc['WEIGHT_SYSTOT']*Tc['WEIGHT_FKP']*Tc['WEIGHT_NOZ']
Dc = cosmo.comoving_distance(z).value

In [315]:
binsfile = np.linspace(1e-5,200+1e-5,41)
mubin = np.linspace(0,1,101)

In [397]:
autocorr=1
DD = DDsmu_mocks(autocorr,1,4,1,100,binsfile,
                        ra,dec,Dc,weights1=w, weight_type='pair_product',is_comoving_dist=True,
                         output_savg=True)

In [476]:
DD

array([(1.0000000e-05,   5.00001,   4.40833183, 0.01,    4, 0.21449457),
       (1.0000000e-05,   5.00001,   4.50050264, 0.02,    6, 0.18603831),
       (1.0000000e-05,   5.00001,   3.93546329, 0.03,    6, 0.1182275 ),
       ...,
       (1.9500001e+02, 200.00001, 197.51515082, 0.98, 1406, 0.17063545),
       (1.9500001e+02, 200.00001, 197.57671282, 0.99, 1772, 0.17173181),
       (1.9500001e+02, 200.00001, 197.53001243, 1.  , 2714, 0.17190348)],
      dtype=[('smin', '<f8'), ('smax', '<f8'), ('savg', '<f8'), ('mumax', '<f8'), ('npairs', '<u8'), ('weightavg', '<f8')])

In [477]:
pairs = DD['npairs']*DD['weightavg']

In [478]:
pairs = pairs.reshape((40,100))

In [479]:
pairs

array([[8.57978297e-01, 1.11622985e+00, 7.09364971e-01, ...,
        4.83881842e-01, 2.94133561e-01, 8.98513917e-01],
       [2.24921781e+00, 3.44164845e+00, 2.66657226e+00, ...,
        2.01080266e-01, 2.75415136e+00, 2.24248297e+00],
       [3.93322622e+00, 4.96740343e+00, 3.89796251e+00, ...,
        5.43156421e+00, 4.24210528e+00, 2.91225960e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.28459101e+02, 3.40286062e+02, 4.35821479e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.16491976e+02, 2.93448011e+02, 4.84230756e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.39913440e+02, 3.04308766e+02, 4.66546041e+02]])

In [321]:
x = Dc*cos(dec*np.pi/180)*cos(ra*np.pi/180)
y = Dc*cos(dec*np.pi/180)*sin(ra*np.pi/180)
z = Dc*sin(dec*np.pi/180)

In [509]:
def distance(cent,pos):
    dist = np.sqrt((pos[:,0] - cent[0])**2+(pos[:,1] - cent[1])**2+(pos[:,2] - cent[2])**2)
    return dist
    
def norm(vec):
    return np.sqrt(vec[:,0]**2+vec[:,1]**2+vec[:,2]**2)

In [540]:
r = np.column_stack([x,y,z])
los = (r+r[0])/2.
ss = (r-r[0])
los = los/norm(los)[:,None]
ss = ss/norm(ss)[:,None]

s = distance(r[0],r)
#mu = np.array(list(map(np.dot,los,ss)))

mu = np.array([np.dot(x,y) for x,y in zip(los,ss)])

/var/folders/3z/9mwhgs_j71j38m0rkc9_lg1r0000gn/T/ipykernel_2336/3214437453.py:5: RuntimeWarning: invalid value encountered in true_divide
  ss = ss/norm(ss)[:,None]


In [541]:
hist,_,_=np.histogram2d(s, mu, bins=(binsfile, mubin))

In [502]:
DDsmu += hist

In [568]:

def DD_brute_smu(*args,**qwargs):
    ra = args[0]
    dec = args[1]
    Dc = args[2]
    w =args[3]
    bins = args[4]
    nmu = args[5]
    mu = np.linspace(0,1,nmu+1)
    x = Dc*cos(dec*np.pi/180)*cos(ra*np.pi/180)
    y = Dc*cos(dec*np.pi/180)*sin(ra*np.pi/180)
    z = Dc*sin(dec*np.pi/180)
    r = np.column_stack([x,y,z])
    size = x.size
        
    count_smu = np.zeros((len(binsfile)-1,len(mubin)-1))
    normDD = 0
    
    for i in range(0,len(x)):
        cent = np.array([x[i],y[i],z[i]])
        wcent = w[i]
        wi = w[int(i+1):size]*wcent
        ri = r[int(i+1):size]
        s = distance(cent,ri)
        los = (cent + ri)/2.
        ss = (cent - ri)
        los = los/norm(los)[:,None]
        ss = ss/norm(ss)[:,None]
        mu = abs(np.array([np.dot(x,y) for x,y in zip(los,ss)]))
        hist,_,_=np.histogram2d(s, mu, bins=(binsfile, mubin),weights=wi)
        count_smu += hist
        normDD += np.sum(wi)
        
    return normDD,count_smu






def DD_brute_rppi(*args,**qwargs):
    ra = args[0]
    dec = args[1]
    Dc = args[2]
    w =args[3]
    bins = args[4]
    nmu = args[5]
    mu = np.linspace(0,1,nmu+1)
    x = Dc*cos(dec*np.pi/180)*cos(ra*np.pi/180)
    y = Dc*cos(dec*np.pi/180)*sin(ra*np.pi/180)
    z = Dc*sin(dec*np.pi/180)
    r = np.column_stack([x,y,z])
    size = x.size
        
    count_smu = np.zeros((len(binsfile)-1,len(mubin)-1))
    
    for i in range(0,len(x)):
        cent = np.array([x[i],y[i],z[i]])
        ri = r[int(i+1):size]
        s = distance(cent,ri)
        los = (cent + ri)/2.
        ss = (cent - ri)
        los = los/norm(los)[:,None]
        ss = ss/norm(ss)[:,None]
        mu = abs(np.array([np.dot(xi,yi) for xi,yi in zip(los,ss)]))
        hist,_,_=np.histogram2d(s, mu, bins=(binsfile, mubin))
        count_smu += hist*wcent
        
    return count_smu
        
        
        
        

In [569]:
from itertools import repeat
list(map(pow, range(10), repeat(3,10)))

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

In [570]:
normDD,DDme = DD_brute_smu(ra,dec,Dc,w,binsfile,100)

In [571]:
binsfile = np.linspace(0,200,41)

In [572]:
dxc = np.diff(binsfile)[0];   xc = binsfile[0:-1] + 0.5*dxc